# Makeheap

1) Реализуйте функцию makeheap_n_log_n(arr), которая преобразует произвольный массив arr в minheap за O(N log N).

2) Реализуйте функцию makeheap(arr), которая преобразует произвольный массив arr в minheap за O(N).

К решению необходимо прикрепить отчет/фото, в котором будет объяснен подход и доказана сложность (подсказка: необходимо аккуратно выписать сумму).

Важно! Нельзя использовать встроенные библиотеки. 

В финале -- сравните время работы двух методов. 

Итог, на выходе этой задачи:

- две функции makeheap_n_log_n(arr), makeheap(arr)

- файл с доказательством асимптотической сложности для makeheap 

- тесты

- сравнение времени выполнения двух методов (в любом виде)

https://interviewing.io/questions/build-a-max-heap

https://stackoverflow.com/questions/9755721/how-can-building-a-heap-be-on-time-complexity/18742428#18742428

https://www.geeksforgeeks.org/dsa/time-complexity-of-building-a-heap/

# Import

In [16]:
import os

while os.getcwd().split("/")[-1] != "algorithms_python":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [17]:
import random
import time
import numpy as np
from typing import Any, Callable, List, Optional, Union

# Timer

time.perf_counter() vs time.process_time()

https://stackoverflow.com/questions/52222002/what-is-the-difference-between-time-perf-counter-and-time-process-time

In [18]:
def timer(func: Callable) -> Callable:
    def wrapper(*args, **kwargs) -> Any:
        start_time = time.perf_counter()  # можно через time.process_time()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()  # можно через time.process_time()
        execution_time = end_time - start_time
        print(f"Execution time of {func.__name__}: {execution_time:.4f} secs")
        return result

    return wrapper

# draw heap

In [19]:
def draw_heap(arr: List[Any]) -> None:
    """
    Красиво отрисовывает массив как бинарное дерево (heap)
    """
    if not arr:
        print("(empty heap)")
        return
    
    def get_lines(start: int = 0, level: int = 0) -> tuple:
        """Рекурсивно строит линии для отрисовки дерева"""
        if start >= len(arr):
            return [], 0, 0, 0
        
        # Текущий узел
        node_str = str(arr[start])
        width = len(node_str)
        
        # Левый и правый потомки
        left_idx = 2 * start + 1
        right_idx = 2 * start + 2
        
        # Лист (нет потомков)
        if left_idx >= len(arr) and right_idx >= len(arr):
            line = node_str
            return [line], width, 1, width // 2
        
        # Только левый потомок
        if right_idx >= len(arr):
            lines, n, p, x = get_lines(left_idx, level + 1)
            s = node_str
            u = len(s)
            first_line = (x + 1) * " " + (n - x - 1) * "_" + s
            second_line = x * " " + "/" + (n - x - 1 + u) * " "
            shifted_lines = [line + u * " " for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2
        
        # Только правый потомок
        if left_idx >= len(arr):
            lines, n, p, x = get_lines(right_idx, level + 1)
            s = node_str
            u = len(s)
            first_line = s + x * "_" + (n - x) * " "
            second_line = (u + x) * " " + "\\" + (n - x - 1) * " "
            shifted_lines = [u * " " + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2
        
        # Есть оба потомка
        left_lines, left_n, left_p, left_x = get_lines(left_idx, level + 1)
        right_lines, right_n, right_p, right_x = get_lines(right_idx, level + 1)
        
        s = node_str
        u = len(s)
        
        # Первая линия: соединяет текущий узел с потомками
        first_line = (left_x + 1) * " " + (left_n - left_x - 1) * "_" + s + right_x * "_" + (right_n - right_x) * " "
        
        # Вторая линия: соединительные линии к потомкам
        second_line = left_x * " " + "/" + (left_n - left_x - 1 + u + right_x) * " " + "\\" + (right_n - right_x - 1) * " "
        
        # Выравнивание высот поддеревьев
        if left_p < right_p:
            left_lines += [left_n * " "] * (right_p - left_p)
        elif right_p < left_p:
            right_lines += [right_n * " "] * (left_p - right_p)
        
        # Объединяем линии левого и правого поддеревьев
        zipped_lines = zip(left_lines, right_lines)
        lines = [first_line, second_line] + [a + u * " " + b for a, b in zipped_lines]
        
        return lines, left_n + right_n + u, max(left_p, right_p) + 2, left_n + u // 2
    
    lines, *_ = get_lines()
    for line in lines:
        print(line)

# makeheap_n_log_n

> Реализуйте функцию makeheap_n_log_n(arr), которая преобразует произвольный массив arr в minheap за O(N log N).

**Логика:**
- Проходимся по массиву в обратном порядке (обрабатываем сначала "листья" потом уже остальные ноды)
- Делаем рекурсивно **sift up** 

![sift_up](../imgs/makeheap_sift_up.png)

**Доказательство Time Complexity:**
- Проходим по всем элементам следовательно то это как минимум **O(N)**.
- Для каждого элемента делаем рекурсивно **sift up**, т.к. **sift up** идет до самого верха, то кол-во **sift up** в худшем случае (для "листьев") равна "высоте кучи/бинарного дерева", которая равна **O(logN)**
- Соединяем все вместе и получаем проходку по всем элементам и для каждого элемента рекурсивный **sift up** -> **O(N * logN)**

In [20]:
@timer
def makeheap_n_log_n(arr: List[Union[int, float]]) -> List[Union[int, float]]:
    def sift_up(heap: List[Union[int, float]], index: int) -> None:
        parent_index = (index - 1) // 2

        if index > 0 and heap[parent_index] > heap[index]:
            heap[parent_index], heap[index] = heap[index], heap[parent_index]

            sift_up(heap, parent_index)

    heap = arr.copy()
    for i in range(1, len(heap)):
        sift_up(heap, i)

    return heap


makeheap_n_log_n(arr=[7, 5, 6, 4, 2, 1, 3])

Execution time of makeheap_n_log_n: 0.0000 secs


[1, 4, 2, 7, 5, 6, 3]

In [21]:
draw_heap(makeheap_n_log_n(arr=[7, 5, 6, 4, 2, 1, 3]))

Execution time of makeheap_n_log_n: 0.0000 secs
  _1_  
 /   \ 
 4   2 
/ \ / \
7 5 6 3


In [29]:
makeheap_n_log_n(arr=[7, 5, 6, 4, 2, 1, 3])

Execution time of makeheap_n_log_n: 0.0000 secs


[1, 4, 2, 7, 5, 6, 3]

In [22]:
arr=[7, 5, 6, 4, 2, 1, 3]
for i in range(len(arr) - 1, 0, -1):
    print(i, arr[i])

6 3
5 1
4 2
3 4
2 6
1 5


In [23]:
arr=list(np.random.randint(low=0, high=100, size=18))
print("min value =" ,min(arr))
draw_heap(makeheap_n_log_n(arr))

min value = 16
Execution time of makeheap_n_log_n: 0.0000 secs
              ______16_______       
             /               \      
        ____16___         __18___   
       /         \       /       \  
    __17___     20_     33_     23_ 
   /       \   /   \   /   \   /   \
  33_     24  46  82  35  43  85  50
 /   \   /                          
53  76  48                          


In [24]:
arr=list(np.random.randint(low=-10000, high=10000, size=20_000_000))
print("min value =" ,min(arr))
print("min heap value = ", makeheap_n_log_n(arr)[0])

min value = -10000
Execution time of makeheap_n_log_n: 3.6797 secs
min heap value =  -10000


# makeheap

Реализуйте функцию makeheap(arr), которая преобразует произвольный массив arr в minheap за O(N).

**Логика:**
- Проходимся по массиву в обратном порядке начиная не с "листьев"
- Делаем рекурсивно **sift down** 

![sift_down](../imgs/makeheap_sift_down.png)
![_sift_down_proof](../imgs/makeheap_sift_down_proof.png)

In [25]:
@timer
def makeheap(arr: List[Union[int, float]]) -> List[Union[int, float]]:
    def sift_down(heap: List[Union[int, float]], index: int) -> None:
        if len(heap) <= 1:
            return

        left_idx = 2 * index + 1
        right_idx = 2 * index + 2
        smallest_idx = index

        if left_idx < len(heap) and heap[left_idx] < heap[smallest_idx]:
            smallest_idx = left_idx

        if right_idx < len(heap) and heap[right_idx] < heap[smallest_idx]:
            smallest_idx = right_idx

        if smallest_idx != index:
            heap[smallest_idx], heap[index] = heap[index], heap[smallest_idx]
            sift_down(heap, smallest_idx)

    heap = arr.copy()
    for i in range(len(heap) // 2 - 1, -1, -1):
        sift_down(heap, i)

    return heap


makeheap(arr=[7, 5, 6, 4, 2, 1, 3])

Execution time of makeheap: 0.0000 secs


[1, 2, 3, 4, 5, 6, 7]

In [26]:
draw_heap(makeheap(arr=[7, 5, 6, 4, 2, 1, 3]))

Execution time of makeheap: 0.0000 secs
  _1_  
 /   \ 
 2   3 
/ \ / \
4 5 6 7


In [27]:
arr=list(np.random.randint(low=0, high=100, size=18))
print("min value =" ,min(arr))
draw_heap(makeheap(arr))

min value = 4
Execution time of makeheap: 0.0000 secs
             _____4______       
            /            \      
        ____5__        __4___   
       /       \      /      \  
    __35___    5_    65_    10_ 
   /       \  /  \  /   \  /   \
  36_     70 87 13 97  98 25  68
 /   \   /                      
80  90  77                      


In [28]:
arr=list(np.random.randint(low=-10000, high=10000, size=20_000_000))
print("min value =" ,min(arr))
print("min heap value = ", makeheap(arr)[0])

min value = -10000
Execution time of makeheap: 4.3086 secs
min heap value =  -10000
